# Data processing

### Intro
This code will get the files with information of each stock and do following:
- Compute some technical indicators for each stock. Hence mfi, rsi, P/E and historical market cap
- Join all stocks together in a large datafile

In [1]:
import tqdm
import time
import os
import tqdm
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
eod_path = Path.cwd()/"Eod" #path to create folder where end of day stock prices will be stored
fundamentals_path = Path.cwd()/"Fundamentals"
SingleStock_data = Path.cwd()/"SingleStock_data"

cwd = Path.cwd()

### Calculate MFI

In [8]:
#First to functions to compute gains and losses will be needed
def gain(x):
    return ((x > 0) * x).sum()


def loss(x):
    return ((x < 0) * x).sum()


def mfi(df, n=14):
    high = df["high"]
    low = df["low"]
    close = df["adjusted_close"]
    volume = df["volume"]
    
    typical_price = (high + low + close)/3
    money_flow = typical_price * volume
    
    mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
    signed_mf = money_flow * mf_sign
    
    mf_avg_gain = signed_mf.rolling(n).apply(gain, raw=True)
    mf_avg_loss = signed_mf.rolling(n).apply(loss, raw=True)
    
    mfi = (100 - (100 / (1 + (mf_avg_gain / abs(mf_avg_loss))))).to_numpy()
    
    return mfi


### Calculate volatility

In [9]:
def vol(df, period = 21):
    daily_return = df['adjusted_close'].pct_change()
    
    volatility = daily_return.rolling(window = period).std()
    
    return volatility


In [10]:
def rsi(df, periods = 14):
    
    delta = df['adjusted_close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

def gain(x):
    return ((x > 0) * x).sum()


def loss(x):
    return ((x < 0) * x).sum()


def mfi(df, n=14):
    high = df["high"]
    low = df["low"]
    close = df["adjusted_close"]
    volume = df["volume"]
    
    typical_price = (high + low + close)/3
    money_flow = typical_price * volume
    
    mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
    signed_mf = money_flow * mf_sign
    
    mf_avg_gain = signed_mf.rolling(n).apply(gain, raw=True)
    mf_avg_loss = signed_mf.rolling(n).apply(loss, raw=True)
    
    mfi = (100 - (100 / (1 + (mf_avg_gain / abs(mf_avg_loss))))).to_numpy()
    
    return mfi

def vol(df, period = 21):
    daily_return = df['adjusted_close'].pct_change()
    
    volatility = daily_return.rolling(window = period).std()
    
    return volatility

def compute_technical(df):
    df["date"] =pd.to_datetime(df["date"])
    #compute RSI
    df['RSI'] = rsi(df)
    
    #compute MFI
    df["MFI"] = mfi(df)
    
    #compute volatility
    df["volatility"]=vol(df)
    
    return df
    
    

## Create master dataframe:
Time to join all dataframes into one

In [11]:
def compute_data(df):
    
    df["date"] = pd.to_datetime(df["date"])
    df = compute_technical(df)
    df = df.drop('close',axis =1)
    df["returns"] = (df["adjusted_close"]-df["adjusted_close"].shift(-1))/df["adjusted_close"].shift(-1)*100
    df['prev_returns'] = df['returns'].shift(-1)
    df['returns_pred'] = df['returns'].shift(1)
    df = df[(df["date"]>='2022-06-01')&(df["date"]<='2023-06-02')]
    
    return df

def master_df(master, df, ticker):
    """
    Finds data for a single stocks and joins it to masterdataframe on=date how=left
    
    Args(pandas.DataFrame): dataframe of the company that will be joined to the master dataframe
                            the existing master dataframe
    Args(str): ticker of the company that will be joined to master dataframe
    
    returns the updated master dataframe
    """

    df["date"] = pd.to_datetime(df["date"])
    df["Ticker"] = ticker

    master = pd.concat([master,df])
    master["date"] = pd.to_datetime(master["date"])
    master = master.sort_values(["date", "Ticker"], ascending=[False, True]).reset_index(drop = True)
    
    return master


def compute_fundamentals(df):
    df["date"] = pd.to_datetime(df["date"])
    df["marketcap"] = df["adjusted_close"]*df["outstandingShares"]
    df["PE"] = df["adjusted_close"]/df["epsActual"]
    #RSI missing
    #MFI missing
    #Standard deviation missing
    
    return df

In [13]:
#get tickers for stocks
# tickers = pd.read_csv("Tickers_final.csv")["Ticker"].values
tickers = pd.read_csv("TickerNames_1956.csv")['0'].values
tickers

array(['AAPL', 'MSFT', 'GOOG', ..., 'CURI', 'NSTS', 'STIM'], dtype=object)

### Master dataframe for EoD only

In [14]:
#delete this sentence # delete this if you want to rund the code - its just a fail safe

master_eod = pd.DataFrame() #define empty dataframe. THis is the final dataframe
for ticker in tqdm.tqdm(tickers):
    #get datafram from csv
    df = pd.read_csv(eod_path/f"{ticker}.csv")
    master_eod = master_df(master_eod,df,ticker)
    master_eod.to_csv("Master_eod.csv", index = False)
    

  0%|          | 0/1956 [00:00<?, ?it/s]

 15%|█▌        | 295/1956 [01:41<09:28,  2.92it/s]


KeyboardInterrupt: 

In [168]:
master_eod = master_eod.sort_values("date", ascending=False)

In [45]:
mast = pd.read_csv("Master_eod.csv").drop("Unnamed: 0", axis =1)
mast["date"]=pd.to_datetime(mast["date"])
mast.sort_values(['date', 'Ticker'], ascending=[False,True])

,date,open,high,low,close,adjusted_close,volume,Ticker
421640,2023-06-02,8.01,8.0900,7.87,8.0600,8.0600,74700,AADI
53848,2023-06-02,14.90,15.0700,14.64,14.8700,14.8700,31041300,AAL
296164,2023-06-02,2.21,2.2700,2.16,2.2700,2.2700,148200,AAOI
79756,2023-06-02,88.82,91.6800,88.73,91.5800,91.5033,243389,AAON
0,2023-06-02,181.03,181.7800,179.26,180.9500,180.7060,61945900,AAPL
...,...,...,...,...,...,...,...,...
324103,2022-05-31,33.51,33.5100,32.30,32.8100,32.8100,418900,ZUMZ
398525,2022-05-31,4.72,4.8000,4.58,4.6300,4.6300,355672,ZVRA
290067,2022-05-31,6.89,7.0300,6.60,6.7200,6.7200,3481600,ZYME
484377,2022-05-31,1.02,1.0200,0.94,0.9575,0.9575,218994,ZYNE


### Master dataframe with technicals, returns and taget returns

In [309]:
delete this sentence # delete this sentence if you are sure you want to run this code

master_tech = pd.DataFrame() #define empty dataframe. THis is the final dataframe

for ticker in tqdm.tqdm(tickers):
    #get datafram from csv
    df = pd.read_csv(eod_path/f"{ticker}.csv")
    df = compute_data(df)
    master_tech = master_df(master_tech, df, ticker)
    master_tech.to_csv("Master_with_technicals_news.csv", index = False)

100%|██████████| 1956/1956 [1:34:11<00:00,  2.89s/it]


In [19]:
master_tech = pd.read_csv("master_df.csv")
master_tech = master_tech.sort_values(["date","Ticker"], ascending = [False,True])

In [21]:
master_tech

,date,open,high,low,adjusted_close,volume,RSI,MFI,volatility,returns,prev_returns,returns_pred,Ticker
0,2023-06-02,8.01,8.0900,7.8700,8.0600,74700,66.502362,48.430352,0.029908,2.025316,-1.250000,0.868486,AADI
1,2023-06-02,14.90,15.0700,14.6400,14.8700,31041300,23.119364,35.726831,0.014020,1.156463,-0.541272,-0.403497,AAL
2,2023-06-02,2.21,2.2700,2.1600,2.2700,148200,26.678504,47.236530,0.089979,2.714932,-0.450450,2.643172,AAOI
3,2023-06-02,88.82,91.6800,88.7300,91.5033,243389,36.479318,50.327996,0.016983,4.459876,1.223862,-0.458563,AAON
4,2023-06-02,181.03,181.7800,179.2600,180.7060,61945900,36.976220,45.237000,0.009638,0.477572,1.602217,-0.757141,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494863,2022-06-01,33.26,33.3400,32.2600,32.9700,557900,68.832023,57.020542,0.033060,0.487656,-3.300914,3.639672,ZUMZ
494864,2022-06-01,4.64,4.7050,4.4900,4.5700,257036,45.655482,44.451853,0.035933,-1.295896,-1.279318,3.501094,ZVRA
494865,2022-06-01,6.85,7.1100,6.6450,6.9800,996500,59.232450,56.263023,0.052836,3.869048,-2.183406,-3.295129,ZYME
494866,2022-06-01,0.97,0.9899,0.9021,0.9050,237207,39.897928,87.930630,0.087510,-5.483029,-1.875384,7.182320,ZYNE


In [106]:
# Find the location of NaN values
nan_locations = np.where(pd.isnull(master_tech))

# The 'nan_locations' tuple contains two arrays: one for row indices and one for column indices
row_loc, cols_loc = nan_locations

# after analysing NaN values we notice that the missings values for MFI column are simply 0
# The calculation returns NaN when the high or low price is the same for more than 14 days
#as the average becomes 0 and we end up dividing by zero. Therefore, all NaN in MFI column are filled by zero

master_tech["MFI"].fillna(0, inplace = True)

#For volatility there were only 4 companies missing the last two days of volatility data
#as we wont use the 2nd, we simply fill these values with 
master_tech["volatility"].fillna(method = 'bfill', inplace = True)
master_tech.to_csv("Master_with_technicals_news.csv", index = False)